# Cohere reranker

>[Cohere](https://cohere.ai/about)는 기업이 인간-기계 상호작용을 개선할 수 있도록 돕는 자연어 처리 모델을 제공하는 캐나다의 스타트업입니다.

이 노트북은 `retriever`에서 [Cohere의 rerank endpoint](https://docs.cohere.com/docs/reranking)를 사용하는 방법을 보여줍니다. 

In [1]:
# 설치
!pip install --upgrade pip
!pip install cohere==5.3.2 langchain-cohere==0.1.4 langchain-core langchain pydantic

  Using cached cohere-5.3.2-py3-none-any.whl.metadata (3.1 kB)
  Using cached langchain_cohere-0.1.4-py3-none-any.whl.metadata (6.4 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.24-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.23-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.22-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.21-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.20-py3-none-any.whl.metadata (7.7 kB)
  Using cached langchain-0.3.19-py3-none-any.whl.metadata (7.9 kB)
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with oth

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.2.4 requires langchain-core>=0.3.60, but you have langchain-core 0.1.53 which is incompatible.
langchain-experimental 0.0.61 requires langchain-community<0.3.0,>=0.2.5, but you have langchain-community 0.0.38 which is incompatible.
langchain-experimental 0.0.61 requires langchain-core<0.3.0,>=0.2.7, but you have langchain-core 0.1.53 which is incompatible.
langchain-google-genai 2.1.6 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.
langchain-google-genai 2.1.6 requires langchain-core<0.4.0,>=0.3.66, but you have langchain-core 0.1.53 which is incompatible.
langchain-huggingface 0.3.0 requires langchain-core<1.0.0,>=0.3.65, but you have langchain-core 0.1.53 which is incompatible.
langchain-huggingface 0.3.0 requires to

## Cohere API 키 설정

- [API 키 발급](https://dashboard.cohere.com/api-keys)
- `.env` 파일에 `COHERE_API_KEY` 키 값에 API 키를 넣어주세요.

**참고**
- [공식 도큐먼트](https://docs.cohere.com/docs/the-cohere-platform?_gl=1*st323v*_gcl_au*MTA2ODUyNDMyNy4xNzE4MDMzMjY2*_ga*NTYzNTI5NDYyLjE3MTgwMzMyNjc.*_ga_CRGS116RZS*MTcyMTk4NzMxMi4xMS4xLjE3MjE5ODczNjIuMTAuMC4w)
- [Reranker Model 리스트](https://docs.cohere.com/docs/rerank-2)

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Reranker")

## 사용법

In [3]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

**Cohere 다국어 지원 모델**

- Embedding: `embed-multilingual-v3.0`, `embed-multilingual-light-v3.0`, `embed-multilingual-v2.0`
- Reranker: `rerank-multilingual-v3.0`, `rerank-multilingual-v2.0`

## CohereRerank을 사용한 재정렬
이제 기본 `retriever`를 `ContextualCompressionRetriever`로 감싸보겠습니다. Cohere 재정렬 엔드포인트를 사용하여 반환된 결과를 재정렬하는 `CohereRerank`를 추가할 것입니다.
CohereRerank에서 모델 이름을 지정하는 것이 필수임을 유의하십시오!

In [4]:
# 아래 셀을 반드시 먼저 실행하고 커널을 재시작하세요.
# !pip install --upgrade langchain-core langchain langchain-cohere cohere pydantic

# 이후 셀부터는 패키지 설치 명령 없이 코드만 작성합니다.
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings, CohereRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

# 문서 로드 (인코딩 지정)
documents = TextLoader("./data/appendix-keywords.txt", encoding="utf-8").load()

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 임베딩 및 벡터스토어 생성
embeddings = CohereEmbeddings(model="embed-multilingual-v3.0")
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# Cohere Reranker 설정
reranker = CohereRerank(model="rerank-multilingual-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker, base_retriever=retriever
)

# 질의문
query = "Word2Vec 에 대해서 알려줘!"

# 재정렬된 문서 검색
reranked_docs = compression_retriever.invoke(query)

# 결과 출력 함수
def pretty_print_docs(docs):
    for i, d in enumerate(docs):
        print(f"\n{'-'*40}\nDocument {i+1}:\n{d.page_content}")

pretty_print_docs(reranked_docs)


----------------------------------------
Document 1:
Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

----------------------------------------
Document 2:
Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

Tokenizer

정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

VectorStore

정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.
예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.
연관키워드: 임베딩, 데이터베이스, 벡터화

SQL

-------